The problem is to identify the Zodiac Sign from the conversation with the user, using Python Chabot approach.

### Below is the code for the Python Chatbot

In [ ]:
# Import Libraries
import json
import random
import os
import re
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
import pandas as pd
import pandas as pd
import datetime
import csv
import gensim
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from statistics import mode 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

Class Objects
*   Contexts - For differentiating the phases during the chat conversation, it’s state should be stored. It can either be flags like “Ordering book” or parameters like “Restaurant: ‘McDonalds’”. With context, you can easily relate intents without knowing what the previous question was. So contexts can be thought of as  flags that are switched on to recognize and maintain relevance to current phase of conversation or switched off to change or exit the phase of conversation. These could also be used as conditional if-else statements. Below are the functions:

    - activate_context( ) and

    - deactivate_context( ) 

* We also have classes like 

    - FirstGreeting( ), 
    - IntentComplete( ) etc..



    

In [ ]:
class Context(object):

	def __init__(self,name):
		self.lifespan = 2
		self.name = name
		self.active = False

	def activate_context(self):
		self.active = True

	def deactivate_context(self):
		self.active = False

		def decrease_lifespan(self):
			self.lifespan -= 1
			if self.lifespan==0:
				self.deactivate_context()

class FirstGreeting(Context): 

	def __init__(self):
		self.lifespan = 1
		self.name = 'FirstGreeting'
		self.active = True

class IntentComplete(Context):

	def __init__(self):
		self.lifespan = 1
		self.name = 'IntentComplete'
		self.active = True


Intents - We have a class for Intent object. For every Intent we have it's name and particular action and parameters. For parameters we load a 'Parameter' object and it has name, prompts, placeholder etc...

In [ ]:
from abc import ABCMeta, abstractmethod
class Intent(object):	


	def __init__(self, name, params, action):
		self.name = name
		self.action = action
		self.params = []
		for param in params:
			# print param['required']
			self.params += [Parameter(param)]

class Parameter():
	def __init__(self, info):
		self.name = info['name']
		self.placeholder = info['placeholder']
		self.prompts = info['prompts']
		self.required = info['required']
		self.context = info['context']

Created .dat files of entities and Intent in the respective folders. Also updated configuration file in the params folder and csv file. 

Run the below code to download the Horoscope intents (called as Utterances in Alexa chatbot), entites (called as Slots in Alexa chatbot), param (config file) and the csv file which contain the possible combinations  to identify the horoscope based on the birthday given

In [ ]:
#@title Run this cell to complete the set up
!wget https://cdn.talentsprint.com/aiml/Hackathon2/chatbot_Horoscope.zip
!unzip chatbot_Horoscope.zip

--2020-05-15 15:42:50--  https://cdn.talentsprint.com/aiml/Hackathon2/chatbot_Horoscope.zip
Resolving cdn.talentsprint.com (cdn.talentsprint.com)... 172.105.34.236
Connecting to cdn.talentsprint.com (cdn.talentsprint.com)|172.105.34.236|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2641 (2.6K) [application/zip]
Saving to: ‘chatbot_Horoscope.zip’

chatbot_Horoscope.z 100%[===================>]   2.58K  --.-KB/s    in 0s      

2020-05-15 15:42:51 (291 MB/s) - ‘chatbot_Horoscope.zip’ saved [2641/2641]

Archive:  chatbot_Horoscope.zip
   creating: chatbot_Horoscope/
   creating: chatbot_Horoscope/entities/
  inflating: chatbot_Horoscope/entities/day.dat  
  inflating: chatbot_Horoscope/entities/month.dat  
  inflating: chatbot_Horoscope/entities/year.dat  
  inflating: chatbot_Horoscope/Horoscope.csv  
   creating: chatbot_Horoscope/intents/
  inflating: chatbot_Horoscope/intents/get_Horoscope.dat  
   creating: chatbot_Horoscope/params/
  inflating: chatbot_Ho

In [ ]:
path_param = '/content/chatbot_Horoscope/params/newparams.cfg'
path_intent= '/content/chatbot_Horoscope/intents/'
path_entities= '/content/chatbot_Horoscope/entities/'
path_csv = '/content/chatbot_Horoscope/Horoscope.csv'

Text Representation

Generate ngrams and Perform precision recall to get accuracy for matching ngrams. 


In [ ]:
# Generate ngrams and Perform precision recall to get accuracy for matching ngrams.

limit = 3

def generatengrams(lines):
    global limit
    ngrams = []
    for i in range(1, limit+1):
        ndict = {}
        for line in lines:
            nline = ['<START>']*i + line + ['<END>']*i
            for x in range(len(nline)- i) :
                key = '_'.join(nline[x:x+i])
                if key in ndict.keys():
                    ndict[key] += 1
                else:
                    ndict[key] = 1
        ngrams += [ndict]
    return ngrams

#Splitting the line by ignoring new line character at the end
def cleanLines(lines):
    for i in range(len(lines)):
        lines[i] = lines[i][:-1].split()
        for x in range(len(lines[i])):
            lines[i][x] = lines[i][x].lower()
    return lines

#Initializing Ngrams dictionary for provided intents
def init():
    ngramsdict = {}
    path = path_intent

    for fil in os.listdir(path):
        if fil.endswith('.dat'):	
            with open(path + fil) as f:
                lines = f.readlines()
                lines = cleanLines(lines)
                ngramsdict[''.join(fil.split('.')[:-1])] = generatengrams(lines)
    return ngramsdict

#ngrammatch takes user input and returns score by comparing input with intents provided
def ngrammatch(uinput, BotsGramsdict=init()):
    global limit
    scores = []
    uinput = [uinput.lower().split()]
    user_ngramsdict = generatengrams(uinput)
    for key in BotsGramsdict:
        ngramsdict = BotsGramsdict[key]
        fscore = 0.0
        for i in range(len(user_ngramsdict)):
            cur_dict = user_ngramsdict[i]
            ansdict = ngramsdict[i]
            precision = 0	
            for i in cur_dict.keys():
                if i in ansdict.keys():
                    precision+=1
            recall = 0
            for i in ansdict.keys():
                if i in cur_dict.keys():
                    recall+=1
            fscore += 1.0/float((len(ansdict.keys())/float(precision) + len(ansdict.keys())/float(recall)))
        scores+= [(key,fscore)]
    return scores

Defining functions for Loading Intent, Collecting params, Checking actions, Identifying Intents and Getting Attributes 

In [ ]:
def loadIntent(path, intent):
    with open(path) as fil:
        dat = json.load(fil)
        intent = dat[intent]
        return Intent(intent['intentname'],intent['Parameters'], intent['actions'])


def check_required_params(current_intent, attributes, context):
    '''Collects attributes pertaining to the current intent'''
    for para in current_intent.params:     	
        if para.required:
            if para.name not in attributes:
                return random.choice(para.prompts), context
    return None, context
    

def check_actions(current_intent, attributes, context):
    '''This function performs the action for the intent as mentioned 
    in the intent config file. Performs actions pertaining to current intent 
    for action in current_intent.actions: 
    if action.contexts_satisfied(active_contexts):
            return perform_action()'''

    context = IntentComplete()
    return current_intent.action, context


def input_processor(user_input, context, attributes, intent):
    '''Update the attributes, abstract over the entities in user input'''
    attributes, cleaned_input = getattributes(user_input, context, attributes)
    return attributes, cleaned_input
    

def intentIdentifier(clean_input, context,current_intent):
    clean_input = clean_input.lower()
    scores = ngrammatch(clean_input)
    
    #choosing here the intent with the highest score
    scores = sorted_by_second = sorted(scores, key=lambda tup: tup[1])   
    if(current_intent==None):
        return loadIntent(path_param,scores[-1][0])
    else:
        #If current intent is not none, stick with the ongoing intent
        return current_intent


def getattributes(uinput,context,attributes):
    '''This function marks the entities in user input, and updates
    the attributes dictionary'''

    if context.name.startswith('IntentComplete'):
        return attributes, uinput
    else:
        #Code can be optimised here, loading the same files each time suboptimal 
        files = os.listdir(path_entities)
        entities = {}
        for fil in files:
            if fil == ".ipynb_checkpoints":
                continue
            lines = open(path_entities+fil).readlines()
            for i, line in enumerate(lines):
                if lines[i].endswith('/n'):
                  lines[i] = line[:-2]
                lines[i]=lines[i].strip()

                if lines[i].lower() in uinput.lower():
                    attributes[fil[:-4]] = lines[i]
                uinput = re.sub(line,r'$'+fil[:-4],uinput,flags=re.IGNORECASE)
        return attributes, uinput                        

Session class is one active session of the chatbot which the user interacts with. Let's go in to the details:

**reply( )** is the important one in our session object it takes user_input as parameter and calls different modules of our chatbot architecture:


*   **input_processor( )** - It helps in preprocessing and fetching the entities that we can identify in the ready state
    
    - **getattributes( )** - It helps in identifying all the entities in the user utterance. We fetch them and map to our parameters
    
    
*   **intentIdentifier( )**

    - **generatengrams** - N-grams are contiguous sequences of n-items in a sentence. When performing machine learning tasks related to natural language processing, we usually need to generate n-grams from input sentences.
     **ngrammatch( )** - generates sequence of words, this algorithm checks for how many unigrams, bigrams, trigrams are common between the utterances for your particular intent and your current user input.
     

*   **check_required_params( )** - Based on the current intents we go over it's parameters

*   **check_actions( )** - This function performs the action for the intent


       


In [ ]:
class Session:
    def __init__(self, attributes=None, active_contexts=[FirstGreeting(), IntentComplete() ]):     
        '''Initialise a default session'''
        #Active contexts not used yet, can use it to have multiple contexts
        self.active_contexts = active_contexts
        
        #Contexts are flags which control dialogue flow       
        self.context = FirstGreeting()
        
        #Intent tracks the current state of dialogue
        self.current_intent = FirstGreeting()
        self.current_intent = None
        
        #attributes hold the information collected over the conversation
        self.attributes = {}
        
    def update_contexts(self):
        '''Not used yet, but is intended to maintain active contexts'''
        for context in self.active_contexts:
            if context.active:
                context.decrease_lifespan()
    global intentCompleted
    def reply(self, user_input):
        '''Generate response to user input'''
        global intentCompleted
        intentCompleted=False
        self.attributes, clean_input = input_processor(user_input, self.context, self.attributes, self.current_intent)

        self.current_intent = intentIdentifier(clean_input, self.context, self.current_intent)

        prompt, self.context = check_required_params(self.current_intent, self.attributes, self.context)

       
        #promp""t being None means all parameters satisfied, perform the intent action
        
        global attr
        if prompt is None:
            if self.context.name!='IntentComplete':
                prompt, self.context = check_actions(self.current_intent, self.attributes, self.context)
                if prompt.endswith('()'):
                  attr=self.attributes
                  prompt=eval(prompt)
        
        #Resets the state after the Intent is complete
        if self.context.name=='IntentComplete':
            self.attributes = {}
            intentCompleted=True # It becomes true when the intent is completed
            self.context = FirstGreeting()
            self.current_intent = None
        return prompt

Run This API Blocks to perform action after satisfying all the attributes specified for particular Intent

In [ ]:
def get_HoroscopeAction():    
  global attr
  print("choosen attributes are",attr)
  year = attr['year']
  month = attr['month'] # month is a string, and we need to convert it to a month index
  day = int(attr['day'])
  
  try:
      month = int(datetime.datetime.strptime(month,'%b').strftime('%m'))
  except:
      month = int(datetime.datetime.strptime(month,'%B').strftime('%m'))
  
  horoscope=""
  usr_dob = (month,day)
  with open(path_csv, 'r') as file:
    reader = csv.reader(file)
    for row in reader:        
        if filter(row[0]) <= usr_dob <= filter(row[1]):
            horoscope=row[2]
  return "Your Horoscope is "+horoscope
   

def filter(X):
    date = X.split()
    try:
      month = int(datetime.datetime.strptime(date[0],'%b').strftime('%m'))
    except:
      month = int(datetime.datetime.strptime(date[0],'%B').strftime('%m'))
    day = int(datetime.datetime.strptime(date[1],'%d').strftime('%d'))
    return (month,day)

###Main Block to access ChatBot
enter 'end' to stop the bot

In [ ]:
print ('BOT: Hi! How may I assist you?')
session=Session()
attr={}
intentCompleted=False
while (True):
    inp = input('User: ')
    prompt=session.reply(inp)
    print ('BOT:', prompt)
    if intentCompleted:
      a=input("BOT: Do you want to proceed (Y/N)?")
      if a.lower().startswith( "n"):          
        print("GoodBye")
        break
      else:
        print("BOT: welcome back..") 

BOT: Hi! How may I assist you?
User: hi
BOT: Which day(date) were you born?
User: 25th october
BOT: What is your year of birth?
User: 1991
choosen attributes are {'month': 'Oct', 'day': '19', 'year': '1991'}
BOT: Your Horoscope is Libra
BOT: Do you want to proceed (Y/N)?y
BOT: welcome back..
User: y
BOT: Which day(date) were you born?
User: fd
BOT: Which day(date) were you born?
User: 25th october
BOT: What is your year of birth?
User: 1991
choosen attributes are {'month': 'Oct', 'day': '19', 'year': '1991'}
BOT: Your Horoscope is Libra
BOT: Do you want to proceed (Y/N)?n
GoodBye
